# OD Data Cleanup 
* In this notebook, I will clean/organize the raw CDC data so that it is organized by state and month 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
od_df = pd.read_csv('../data/data_raw/us_overdose_data.csv')

### Initial data exploration
Steps
* Load in dataframe
* How many rows/columns are there?
* Define column labels with meta data 
* What does each row represent? 
* Is there missing data?

In [3]:
od_df.shape
od_nrows = od_df.shape[0]
od_ncols = od_df.shape[1]
print(f'There are {od_nrows} rows and {od_ncols} columns in the Overdose data frame')

There are 35490 rows and 12 columns in the Overdose data frame


In [4]:
od_df.columns

Index(['State', 'Year', 'Month', 'Period', 'Indicator', 'Data Value',
       'Percent Complete', 'Percent Pending Investigation', 'State Name',
       'Footnote', 'Footnote Symbol', 'Predicted Value'],
      dtype='object')

* State: state abbreviation 
* Year/Month: separated; need to be combined and converted to datetime; already aggregated by month 
* Period: 12 month period, same for all 
* Indicator: CDC's numerical codes for different drug types (summed in "Number of Drug Overdose Deaths")
* Data Value: Number of overdoses (broken down by type of drug)
* Percent Complete: jurisdictions reporting, same for all 
* Percent Pending Investigation: Deaths that are unconfirmed but suspected to be OD; negligible amount 
* Footnote', 'Footnote Symbol', 'Predicted Value': internal system? No meta data explaining this that I could find 


* Missing data - drug type was not specified for some states in 2015

In [5]:
od_df.sample(10)

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
22651,OH,2019,February,12 month-ending,Number of Deaths,"122,776",100,0.015475,Ohio,Underreported due to incomplete data.,*,NaN
801,AL,2015,March,12 month-ending,Number of Drug Overdose Deaths,733,100,0.308997,Alabama,Numbers may differ from published reports usin...,**,758
25706,SC,2017,July,12 month-ending,Cocaine (T40.5),212,100,0.046033,South Carolina,Numbers may differ from published reports usin...,**,214
28504,US,2015,December,12 month-ending,Percent with drugs specified,83.0777416718925,100,0.153227,United States,Numbers may differ from published reports usin...,**,NaN
4622,DC,2017,February,12 month-ending,"Synthetic opioids, excl. methadone (T40.4)",176,100,0.000000,District of Columbia,Numbers may differ from published reports usin...,**,176
8358,IA,2016,December,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",95,100,0.006822,Iowa,Numbers may differ from published reports usin...,**,96
17688,NC,2020,April,12 month-ending,"Opioids (T40.0-T40.4,T40.6)","1,777",100,0.783646,North Carolina,Underreported due to incomplete data.,*,"1,956"
18970,NJ,2015,January,12 month-ending,Natural & semi-synthetic opioids (T40.2),NaN,100,0.259162,New Jersey,Numbers may differ from published reports usin...,**,NaN
20331,NM,2019,July,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",260,100,0.177058,New Mexico,Underreported due to incomplete data.,*,266
25640,SC,2017,April,12 month-ending,"Synthetic opioids, excl. methadone (T40.4)",273,100,0.038043,South Carolina,Numbers may differ from published reports usin...,**,274


Steps:

1. Subset relevent columns 
2. Filter out territories/cities 
3. Put together Month/Year > convert to datetime 
4. Filer relevent rows - only interested in opioids (which includes natural, semi-synthetic, synthetic opioids) and sums

Step 1: Subset columns

In [6]:
cols_to_use = ['State','Month','Year','Indicator','Data Value']
od_df = od_df[cols_to_use]
od_df

,State,Month,Year,Indicator,Data Value
0,AK,April,2015,Heroin (T40.1),NaN
1,AK,April,2015,"Natural, semi-synthetic, & synthetic opioids, ...",NaN
2,AK,April,2015,Cocaine (T40.5),NaN
3,AK,April,2015,Methadone (T40.3),NaN
4,AK,April,2015,Psychostimulants with abuse potential (T43.6),NaN
...,...,...,...,...,...
35485,YC,May,2020,Heroin (T40.1),688
35486,YC,May,2020,Number of Deaths,"80,969"
35487,YC,May,2020,Natural & semi-synthetic opioids (T40.2),337
35488,YC,May,2020,Number of Drug Overdose Deaths,"1,669"


Step 2: filter out territories/cities

In [7]:
row_filter = od_df['State'].isin(['US', 'DC','YC'])
od_df = od_df[-row_filter]
states = od_df['State'].unique()
len(states)


50

Step 3: Reformat to datetime

In [8]:
od_df = od_df.copy()
od_df['Date'] = pd.to_datetime(od_df['Year'].astype(str) + od_df['Month'], format = '%Y%B')
od_df = od_df.set_index(pd.to_datetime(od_df['Date']))
od_df

,State,Month,Year,Indicator,Data Value,Date
Date,,,,,,
2015-04-01,AK,April,2015,Heroin (T40.1),NaN,2015-04-01
2015-04-01,AK,April,2015,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,2015-04-01
2015-04-01,AK,April,2015,Cocaine (T40.5),NaN,2015-04-01
2015-04-01,AK,April,2015,Methadone (T40.3),NaN,2015-04-01
2015-04-01,AK,April,2015,Psychostimulants with abuse potential (T43.6),NaN,2015-04-01
...,...,...,...,...,...,...
2020-05-01,WY,May,2020,Heroin (T40.1),NaN,2020-05-01
2020-05-01,WY,May,2020,Number of Deaths,"4,808",2020-05-01
2020-05-01,WY,May,2020,Natural & semi-synthetic opioids (T40.2),28,2020-05-01


In [9]:
od_df.shape

(33150, 6)

Step 4: Filter relavent rows

In [10]:
row_fitler = od_df['Indicator'].isin(['Opioids (T40.0-T40.4,T40.6)','Number of Drug Overdose Deaths','Number of Deaths'])
od_opioids_df = od_df[row_fitler]
od_opioids_df.head(10)

,State,Month,Year,Indicator,Data Value,Date
Date,,,,,,
2015-04-01,AK,April,2015,Number of Deaths,"4,133",2015-04-01
2015-04-01,AK,April,2015,Number of Drug Overdose Deaths,126,2015-04-01
2015-04-01,AK,April,2015,"Opioids (T40.0-T40.4,T40.6)",NaN,2015-04-01
2015-08-01,AK,August,2015,"Opioids (T40.0-T40.4,T40.6)",NaN,2015-08-01
2015-08-01,AK,August,2015,Number of Deaths,"4,222",2015-08-01
2015-08-01,AK,August,2015,Number of Drug Overdose Deaths,124,2015-08-01
2015-12-01,AK,December,2015,"Opioids (T40.0-T40.4,T40.6)",NaN,2015-12-01
2015-12-01,AK,December,2015,Number of Drug Overdose Deaths,121,2015-12-01
2015-12-01,AK,December,2015,Number of Deaths,"4,193",2015-12-01


In [11]:
od_df.shape, od_opioids_df.shape

((33150, 6), (9100, 6))

In [12]:
od_opioids_df['State'].nunique()

50

In [13]:
cols_to_use = ['State','Indicator','Data Value']
od_cleaned = od_opioids_df[cols_to_use]
od_cleaned

,State,Indicator,Data Value
Date,,,
2015-04-01,AK,Number of Deaths,"4,133"
2015-04-01,AK,Number of Drug Overdose Deaths,126
2015-04-01,AK,"Opioids (T40.0-T40.4,T40.6)",NaN
2015-08-01,AK,"Opioids (T40.0-T40.4,T40.6)",NaN
2015-08-01,AK,Number of Deaths,"4,222"
...,...,...,...
2020-03-01,WY,Number of Drug Overdose Deaths,81
2020-03-01,WY,"Opioids (T40.0-T40.4,T40.6)",45
2020-05-01,WY,"Opioids (T40.0-T40.4,T40.6)",51


Reformat:
Its difficult to get totals for the three indicators I'm interested in, because they are all in rows - I want to have each of them in their own column.
* Number of Deaths: the total number of deaths by any cause
* Number of Drug OVerdose Deaths: the number of deaths where cause of death is overdose
* Opioids: the number of fatal overdoses caused by opioids (as opposed to another drug type, ex. meth)

In [14]:
o_filter = od_cleaned['Indicator'].isin(['Opioids (T40.0-T40.4,T40.6)'])
opioid_df = od_cleaned[o_filter]
opioid_df['Data Value'] = opioid_df['Data Value'].str.replace(',', '').astype(float)
opioid_df = opioid_df.rename(columns={'Data Value': 'Opioid Deaths'})
del opioid_df['Indicator']
opioid_df['State'].nunique()

40

In [15]:
nod_filter = od_cleaned['Indicator'].isin(['Number of Deaths'])
nod_df = od_cleaned[nod_filter]
nod_df['Data Value'] = nod_df['Data Value'].str.replace(',', '').astype(float)
nod_df = nod_df.rename(columns={'Data Value': 'Number of Deaths'})
del nod_df['Indicator']
nod_df['State'].nunique()

50

In [16]:
nodod_filter = od_cleaned['Indicator'].isin(['Number of Drug Overdose Deaths'])
nodod_df = od_cleaned[nodod_filter]
nodod_df['Data Value'] = nodod_df['Data Value'].str.replace(',', '').astype(float)
nodod_df = nodod_df.rename(columns={'Data Value': 'Number of Drug Overdose Deaths'})
del nodod_df['Indicator']
nodod_df['State'].nunique()

50

In [17]:
pd.merge?

In [18]:
merged1 = opioid_df.merge(nod_df, on=['Date', 'State'], how='outer')
merged_df = merged1.merge(nodod_df, on=['Date', 'State'], how='outer')
merged_df['State'].nunique()

50

In [19]:
merged_df=merged_df.sort_values(['State', 'Date'])
merged_df

,State,Opioid Deaths,Number of Deaths,Number of Drug Overdose Deaths
Date,,,,
2015-01-01,AK,NaN,4034.0,126.0
2015-02-01,AK,NaN,4084.0,127.0
2015-03-01,AK,NaN,4101.0,125.0
2015-04-01,AK,NaN,4133.0,126.0
2015-05-01,AK,NaN,4196.0,125.0
...,...,...,...,...
2020-01-01,WY,47.0,4748.0,78.0
2020-02-01,WY,46.0,4783.0,80.0
2020-03-01,WY,45.0,4757.0,81.0


In [20]:
merged_df['State'].nunique()

50

In [21]:
merged_df.reset_index(level=0,inplace=True)
merged_df

,Date,State,Opioid Deaths,Number of Deaths,Number of Drug Overdose Deaths
0,2015-01-01,AK,NaN,4034.0,126.0
1,2015-02-01,AK,NaN,4084.0,127.0
2,2015-03-01,AK,NaN,4101.0,125.0
3,2015-04-01,AK,NaN,4133.0,126.0
4,2015-05-01,AK,NaN,4196.0,125.0
...,...,...,...,...,...
3245,2020-01-01,WY,47.0,4748.0,78.0
3246,2020-02-01,WY,46.0,4783.0,80.0
3247,2020-03-01,WY,45.0,4757.0,81.0
3248,2020-04-01,WY,46.0,4828.0,83.0


In [22]:
merged_df.to_csv('../data/od_data_longterm.csv',index=False)